In [15]:
import pendulum
import pandas as pd
from sqlalchemy import create_engine, MetaData
from sqlalchemy import select


PG_USER = "postgres"
PG_PASSWORD = "data123"
PG_HOSTNAME = "127.0.0.1"
PG_PORT = "5432"
PG_DB_NAME = "dailypricedata"

connect = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOSTNAME}:{PG_PORT}/{PG_DB_NAME}"

engine = create_engine(connect)


In [16]:
meta_data = MetaData(bind=engine)
MetaData.reflect(meta_data)
nsedata = meta_data.tables['NSEData']
query = select([
    nsedata.c.Symbol
])

stockNameList = engine.execute(query).fetchall()

In [17]:

stockLastUpdateDict = {}
# Exception list BAYERCROP, ABBOTINDIA, HLEGLAS, MCX, PCBL, RBA, SPICEJET, WESTLIFE, ZFCVINDIA, AGI, BESTAGRO, COSMOFIRST, DOLATALGO, FORCEMOT, GATEWAY, GOODYEAR, HINDWAREAP, IONEXCHANG, KENNAMET, KIRLFER
for stockName in stockNameList:
    try:
        stock = meta_data.tables[stockName[0]]
    except:
        print(stockName[0])
    query = select([
        stock.c.Date
    ])
    dateRange = engine.execute(query).fetchall()
    df = pd.DataFrame(dateRange)
    if df.empty:
        continue
    latestDataDate = pendulum.parse(str(df.tail(1).values[0][0]))
    stockLastUpdateDict[stockName[0]] = latestDataDate
#print(stockLastUpdateDict)
    


In [18]:
def to_alchemy(anydbengine, stockname, dataframe):
    """dumps data frame to database"""
    dataframe.to_sql(
        stockname,
        con=anydbengine,
        index=False,
        if_exists='append'
    )

In [ ]:
from datetime import datetime
import yfinance as yf
stockUpdateStartDate = stockLastUpdateDict
endDate = datetime(2022, 6, 1)
for stockname in stockUpdateStartDate:
    startDate = stockUpdateStartDate[stockname]
    startDate = pendulum.instance(startDate).add(days = 1)
    print(stockname,startDate, endDate)
    stock_price_info_Frame = yf.download(
        stockname + ".NS", start=startDate, end=endDate, progress=False).reset_index()
    to_alchemy(engine, stockname, stock_price_info_Frame)
